# I fucking hate `pandas` and `sklearn` for their messy and inconsistent interfaces
# for their opposition to any extensions of their libraries
# and all that crap.
# I can't even sleep because of how much I hate them.
# I was trying to write things according to their API
# implementing BaseEstimator and TransformerMixin
# and using pipes
# but that makes things even worser
# because of uglyness of this environment!

# I wanted to try pretty obvious tricks and models
# but this environment makes it impossible

In [1]:
import pandas as pd
import numpy as np
import dateutil.parser
import datetime
import sklearn.preprocessing
import collections
from sklearn.preprocessing import (
    StandardScaler,
    PolynomialFeatures
)
from sklearn.feature_extraction.text import (
    CountVectorizer
)
from sklearn.feature_selection import (
    SelectKBest, SelectPercentile,
    SelectFpr, VarianceThreshold,
    SelectFromModel,
    chi2, f_classif,
)
from sklearn.ensemble import (
    RandomForestClassifier
)
from sklearn.linear_model import (
    LogisticRegression
)
from sklearn.pipeline import (
    FeatureUnion, Pipeline
)
from scipy.sparse import hstack


%matplotlib inline

In [2]:
X_train = pd.read_csv('X_train.csv')
y_train = pd.read_csv('y_train.csv')
X_test = pd.read_csv('X_test.csv')

In [3]:
X_train.dtypes

id                         int64
amount_tsh               float64
date_recorded             object
funder                    object
gps_height                 int64
installer                 object
longitude                float64
latitude                 float64
wpt_name                  object
num_private                int64
basin                     object
subvillage                object
region                    object
region_code                int64
district_code              int64
lga                       object
ward                      object
population                 int64
public_meeting            object
recorded_by               object
scheme_management         object
scheme_name               object
permit                    object
construction_year          int64
extraction_type           object
extraction_type_group     object
extraction_type_class     object
management                object
management_group          object
payment                   object
payment_ty

In [4]:
display(
    X_train.isnull().sum()[X_train.isnull().sum() > 0],
    X_test.isnull().sum()[X_test.isnull().sum() > 0]
)

funder                3635
installer             3655
subvillage             371
public_meeting        3334
scheme_management     3877
scheme_name          28166
permit                3056
dtype: int64

funder                869
installer             877
subvillage             99
public_meeting        821
scheme_management     969
scheme_name          7092
permit                737
dtype: int64

In [5]:
def date_to_ymd(x):
    res = (np.nan, np.nan, np.nan)
    if isinstance(x, str):
        try:
            x = dateutil.parser.parse(x)
        except:
            pass
    if isinstance(x, datetime.date):
            res = (x.year, x.month, x.day)
    return res

In [6]:
X_train.drop(['id', 'recorded_by', 'num_private'], axis=1, inplace=True)
X_test.drop(['id', 'recorded_by', 'num_private'], axis=1, inplace=True)

In [7]:
(X_train.loc[:,'date_recorded_year'],
 X_train.loc[:,'date_recorded_month'],
 X_train.loc[:,'date_recorded_day']) = zip(*X_train.loc[:,'date_recorded'].map(date_to_ymd))
(X_test.loc[:,'date_recorded_year'],
 X_test.loc[:,'date_recorded_month'],
 X_test.loc[:,'date_recorded_day']) = zip(*X_test.loc[:,'date_recorded'].map(date_to_ymd))

X_train.drop('date_recorded', axis=1, inplace=True)
X_test.drop('date_recorded', axis=1, inplace=True)

X_train.loc[:, 'dry_season'] = 1*(X_train.loc[:, 'date_recorded_month'].isin([11, 12, 1, 2, 3, 4]))
X_train.loc[:, 'rainy_season'] = 1*(X_train.loc[:, 'date_recorded_month'].isin([5,6,9,10]))
X_test.loc[:, 'dry_season'] = 1*(X_train.loc[:, 'date_recorded_month'].isin([11, 12, 1, 2, 3, 4]))
X_test.loc[:, 'rainy_season'] = 1*(X_train.loc[:, 'date_recorded_month'].isin([5,6,9,10]))


X_train.loc[:, 'date_recorded_month'] /= 12
X_test.loc[:, 'date_recorded_month'] /= 12

In [8]:
X_train.loc[:,'age'] = X_train.loc[:,'date_recorded_year'] - X_train.loc[:,'construction_year']
X_test.loc[:,'age'] = X_test.loc[:,'date_recorded_year'] - X_test.loc[:,'construction_year']

In [9]:
X_train.drop(['date_recorded_year', 'date_recorded_day'], axis=1, errors='ignore', inplace=True)
X_test.drop(['date_recorded_year', 'date_recorded_day'], axis=1, errors='ignore', inplace=True)

In [10]:
heightscale = StandardScaler().fit(pd.concat((X_train.loc[:, 'gps_height'],
                                              X_test.loc[:, 'gps_height']), ignore_index=True))
X_train.loc[:, 'gps_height'] = heightscale.transform(X_train.loc[:, 'gps_height']).reshape(-1, 1)
X_test.loc[:, 'gps_height'] = heightscale.transform(X_test.loc[:, 'gps_height']).reshape(-1, 1)

/home/god/py/lib/python3.6/site-packages/sklearn/utils/validation.py:429: DataConversionWarning: Data with input dtype int64 was converted to float64 by StandardScaler.
  warnings.warn(msg, _DataConversionWarning)
/home/god/py/lib/python3.6/site-packages/sklearn/preprocessing/data.py:586: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  warnings.warn(DEPRECATION_MSG_1D, DeprecationWarning)
/home/god/py/lib/python3.6/site-packages/sklearn/preprocessing/data.py:649: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  warnings.warn(DEPRECATION_MSG_1D, DeprecationWarning)
/home/god/py/lib/python3.6/site-packages/sklearn

In [11]:
X_train.loc[:,'amount_per_man'] = (X_train.loc[:,'amount_tsh']/X_train.loc[:,'population'])
X_test.loc[:,'amount_per_man'] = (X_test.loc[:,'amount_tsh']/X_test.loc[:,'population'])
X_train.loc[:,'amount_per_man'].replace([np.inf, -np.inf, np.nan], 0, inplace=True)
X_test.loc[:,'amount_per_man'].replace([np.inf, -np.inf, np.nan], 0, inplace=True)
# display(X_test.loc[:,'amount_per_man'])
amntpmscaler = StandardScaler().fit(X_test.loc[:, 'amount_per_man'])
X_train.loc[:,'amount_per_man'] = amntpmscaler.transform(X_train.loc[:,'amount_per_man']).reshape(-1, 1)
X_test.loc[:,'amount_per_man'] = amntpmscaler.transform(X_test.loc[:,'amount_per_man']).reshape(-1, 1)

/home/god/py/lib/python3.6/site-packages/sklearn/preprocessing/data.py:586: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  warnings.warn(DEPRECATION_MSG_1D, DeprecationWarning)
/home/god/py/lib/python3.6/site-packages/sklearn/preprocessing/data.py:649: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  warnings.warn(DEPRECATION_MSG_1D, DeprecationWarning)
/home/god/py/lib/python3.6/site-packages/sklearn/preprocessing/data.py:649: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature o

In [ ]:
X_train.loc[:,'population'] /= X_test.loc[:,'population'].max()
X_test.loc[:,'population'] /= X_test.loc[:,'population'].max()

In [12]:
amounttshscaler = StandardScaler().fit(pd.concat((X_train.loc[:, 'amount_tsh'],
                                              X_test.loc[:, 'amount_tsh']), ignore_index=True))
X_train.loc[:, 'amount_tsh'] = amounttshscaler.transform(X_train.loc[:, 'amount_tsh']).reshape(-1, 1)
X_test.loc[:, 'amount_tsh'] = amounttshscaler.transform(X_test.loc[:, 'amount_tsh']).reshape(-1, 1)

/home/god/py/lib/python3.6/site-packages/sklearn/preprocessing/data.py:586: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  warnings.warn(DEPRECATION_MSG_1D, DeprecationWarning)
/home/god/py/lib/python3.6/site-packages/sklearn/preprocessing/data.py:649: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  warnings.warn(DEPRECATION_MSG_1D, DeprecationWarning)
/home/god/py/lib/python3.6/site-packages/sklearn/preprocessing/data.py:649: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature o

In [13]:
X_train.loc[X_train.loc[:,'permit'] == True, 'permit'] = 1
X_train.loc[X_train.loc[:,'permit'] == False, 'permit'] = 0
X_train.loc[:,'permit'].fillna(0.5, inplace=True)
X_test.loc[X_test.loc[:,'permit'] == True, 'permit'] = 1
X_test.loc[X_test.loc[:,'permit'] == False, 'permit'] = 0
X_test.loc[:,'permit'].fillna(0.5, inplace=True)

In [14]:
X_train.loc[X_train.loc[:,'public_meeting'] == True, 'public_meeting'] = 1
X_train.loc[X_train.loc[:,'public_meeting'] == False, 'public_meeting'] = 0
X_train.loc[:,'public_meeting'].fillna(0.5, inplace=True)
X_test.loc[X_test.loc[:,'public_meeting'] == True, 'public_meeting'] = 1
X_test.loc[X_test.loc[:,'public_meeting'] == False, 'public_meeting'] = 0
X_test.loc[:,'public_meeting'].fillna(0.5, inplace=True)

In [15]:
X_train.loc[:,'wpt_name'] = X_train.loc[:,'wpt_name'].notnull() * 1
X_test.loc[:,'wpt_name'] = X_test.loc[:,'wpt_name'].notnull() * 1

In [16]:
X_train.loc[:,'source'] = (
    X_train.loc[:,'source'] + ' '
    + X_train.loc[:,'source_type'] + ' '
    + X_train.loc[:,'source_class']
)
X_test.loc[:,'source'] = (
    X_test.loc[:,'source'] + ' '
    + X_test.loc[:,'source_type'] + ' '
    + X_test.loc[:,'source_class']
)

In [17]:
X_train.loc[:,'quality'] = (
    X_train.loc[:,'water_quality'] + ' '
    + X_train.loc[:,'quality_group']
)
X_test.loc[:,'quality'] = (
    X_test.loc[:,'water_quality'] + ' '
    + X_test.loc[:,'quality_group']
)

In [18]:
X_train.loc[:,'waterpoint_type'] = (
    X_train.loc[:,'waterpoint_type'] + ' '
    + X_train.loc[:,'waterpoint_type_group']
)
X_test.loc[:,'waterpoint_type'] = (
    X_test.loc[:,'waterpoint_type'] + ' '
    + X_test.loc[:,'waterpoint_type_group']
)

In [19]:
X_train.loc[:,'quantity'] = (
    X_train.loc[:,'quantity'] + ' '
    + X_train.loc[:,'quantity_group']
)
X_test.loc[:,'quantity'] = (
    X_test.loc[:,'quantity'] + ' '
    + X_test.loc[:,'quantity_group']
)

In [20]:
X_train.loc[:,'management'] = (
    X_train.loc[:,'management'] + ' '
    + X_train.loc[:,'management_group']
)
X_test.loc[:,'management'] = (
    X_test.loc[:,'management'] + ' '
    + X_test.loc[:,'management_group']
)

In [21]:
X_train.loc[:,'payment'] = (
    X_train.loc[:,'payment'] + ' '
    + X_train.loc[:,'payment_type']
)
X_test.loc[:,'payment'] = (
    X_test.loc[:,'payment'] + ' '
    + X_test.loc[:,'payment_type']
)

In [22]:
X_train.loc[:,'extraction_type'] = (
    X_train.loc[:,'extraction_type'] + ' '
    + X_train.loc[:,'extraction_type_group'] + ' '
    + X_train.loc[:,'extraction_type_class']
)
X_test.loc[:,'extraction_type'] = (
    X_test.loc[:,'extraction_type'] + ' '
    + X_test.loc[:,'extraction_type_group'] + ' '
    + X_test.loc[:,'extraction_type_class']
)

In [23]:
drop_cats = ['source_type', 'source_class',
             'water_quality', 'quality_group',
             'quantity_group', 'waterpoint_type_group',
             'payment_type', 'extraction_type_group',
             'extraction_type_class', 'management_group',
             'construction_year', 'date_recorded_month'
]
X_train.drop(drop_cats, axis=1, inplace=True, errors='ignore')
X_test.drop(drop_cats, axis=1, inplace=True, errors='ignore')

In [24]:
X_train.iloc[0,:]

amount_tsh                                         1.95453
funder                                               Roman
gps_height                                         1.04558
installer                                            Roman
longitude                                          34.9381
latitude                                          -9.85632
wpt_name                                                 1
basin                                           Lake Nyasa
subvillage                                        Mnyusi B
region                                              Iringa
region_code                                             11
district_code                                            5
lga                                                 Ludewa
ward                                              Mundindi
population                                             109
public_meeting                                           1
scheme_management                                      V

In [25]:
text_features = ['funder', 'installer', 'basin',
                'subvillage', 'region', 'region_code', 'district_code',
                'lga', 'ward', 
                 'quality',
                 #'water_quality', 'quality_group',
                'scheme_management', 'scheme_name',
                'extraction_type', #'extraction_type_group',
                'management',# 'management_group', 
                 'payment',# 'payment_type',
                'quantity',# 'quantity_group',
                'source',# 'source_type', 'source_class',
                 'waterpoint_type', #'waterpoint_type_group',
                #'date_recorded_year', 'construction_year',
               ]
text_features_vectorizers = dict()
text_feats_train = []
text_feats_test = []
text_selfeat_names = []

In [26]:
y_enc1 = sklearn.preprocessing.LabelEncoder().fit(y_train.iloc[:,1])
ytr = y_enc1.transform(y_train.iloc[:,1])

In [27]:
# lonlat shall be clusterized before usage
# day probably doesn't matter, but month might be (seasonality)
# the years in test and train do not intersect
X_train.drop(['longitude', 'latitude', 'date_recorded_day', 'date_recorded_year'], axis=1, inplace=True, errors='ignore')
X_test.drop(['longitude', 'latitude', 'date_recorded_day', 'date_recorded_year'], axis=1, inplace=True, errors='ignore')

In [28]:
display(X_train.shape, X_test.shape)

(59400, 28)

(14850, 28)

In [29]:
sel = Pipeline([
    # ('varthres', VarianceThreshold(1e-5)),
    ('chi2', SelectPercentile(chi2, 40)),
    #('interact', PolynomialFeatures(2, interaction_only=True)),
    #('feats', FeatureUnion([
        # ('anova', SelectFpr(f_classif, 1e-19)), # yeah, i'm gonna be very pretentious!
        # ('chi2', SelectKBest(chi2, min(128, tr.shape[1])).fit(tr, ytr),)
        # OHE-encoded features are supposed to have additive impact, so linear model:
    ('main', SelectFromModel(RandomForestClassifier(n_estimators=100, max_depth=32)))
    #])),
])
for cat in text_features:
    if X_train.loc[:,cat].dtype == object:
        X_train.loc[:,cat].fillna('unknown', inplace=True)
        X_train.loc[X_train.loc[:,cat].str.contains('other') == True, cat] = 'unknown'
        X_test.loc[:,cat].fillna('unknown', inplace=True)
        X_test.loc[X_test.loc[:,cat].str.contains('other') == True, cat] = 'unknown'
    else:
        X_train.loc[:,cat].fillna(0, inplace=True)
        X_test.loc[:,cat].fillna(0, inplace=True)
        X_train.loc[:cat] = X_train.loc[:cat].astype(str)
        X_test.loc[:cat] = X_test.loc[:cat].astype(str)
    v = CountVectorizer(ngram_range=(1,2), binary=True)
    v.fit(X_test.loc[:,cat])
    tr = v.transform(X_train.loc[:,cat])
    te = v.transform(X_test.loc[:,cat])
    #kbest = SelectKBest(chi2, min(8, tr.shape[1])).fit(tr, ytr)

    sel.fit(tr, ytr)
    tr = sel.transform(tr)
    te = sel.transform(te)
    for i in range(tr.shape[1]):
        text_selfeat_names.append('%s_%s' % (cat, i))
    #tr = pd.DataFrame(tr.toarray()).add_prefix(cat + '_')
    #te = pd.DataFrame(te.toarray()).add_prefix(cat + '_')
    if tr.shape[1] > 0:
        text_feats_train.append(tr)
        text_feats_test.append(te)

text_feats_train = hstack(text_feats_train) # pd.DataFrame(hstack(text_feats_train), columns=text_selfeat_names)
text_feats_test = hstack(text_feats_test) # pd.DataFrame(hstack(text_feats_test), columns=text_selfeat_names)
# print(sel.fit_transform(text_feats_train, ytr).toarray().shape)
# sel_text_feats = SelectKBest(chi2, 48).fit(text_feats_train, ytr) # SelectFpr(f_classif, 1e-19)
sel_text_feats = sel
text_feats_train = sel.fit_transform(text_feats_train, ytr)
text_feats_test = sel.transform(text_feats_test)
print('cat features shape: ', text_feats_train.shape)
# print('top catfeats: ', sel.named_steps['linear'].estimator_.coef_[:20] )
del tr, te
X_train = pd.concat([X_train.drop(text_features, axis=1, errors='ignore'),
                     pd.DataFrame(text_feats_train.toarray())],
                    axis=1)
X_test = pd.concat([X_test.drop(text_features, axis=1, errors='ignore'),
                    pd.DataFrame(text_feats_test.toarray())],
                    axis=1)

cat features shape:  (59400, 73)


In [30]:
X_train.head()

,amount_tsh,gps_height,wpt_name,population,public_meeting,permit,dry_season,rainy_season,age,amount_per_man,...,63,64,65,66,67,68,69,70,71,72
0,1.95452941861,1.04558069781,1,109,1.0,0.0,1,0,12,0.031485078672,...,0,0,0,0,1,1,1,0,1,0
1,-0.109636705889,1.05857227813,1,280,0.5,1.0,1,0,3,-0.0940907887382,...,0,0,0,0,0,0,0,1,1,0
2,-0.101036013704,0.0293504149544,1,250,1.0,1.0,1,0,4,-0.0938626592458,...,0,0,0,0,0,0,0,1,0,0
3,-0.109636705889,-0.581253860116,1,58,1.0,1.0,1,0,27,-0.0940907887382,...,1,1,0,0,0,0,0,0,0,0
4,-0.109636705889,-0.960896707264,1,0,1.0,1.0,0,0,2011,-0.0940907887382,...,0,0,0,0,0,0,0,1,1,0


In [31]:
[(c, X_train[c].dtype) for c in X_train.columns if not np.issubdtype(X_train[c].dtype, np.number)]

[('amount_tsh', dtype('O')),
 ('gps_height', dtype('O')),
 ('wpt_name', dtype('O')),
 ('population', dtype('O')),
 ('public_meeting', dtype('O')),
 ('permit', dtype('O')),
 ('dry_season', dtype('O')),
 ('rainy_season', dtype('O')),
 ('age', dtype('O')),
 ('amount_per_man', dtype('O'))]

In [32]:
def split_train_cv_test(X, y, proportions=(.75, .25/2, .25/2)):
    # expecting $y$ to be numpy array
    outs = np.unique(y)
    proportions = np.array(proportions)
    classes = [np.where(y == i)[0] for i in outs]
    xparts = [[] for _ in proportions]
    yparts = [[] for _ in proportions]
    for cidx in classes:
        cidx = sklearn.utils.shuffle(cidx)
        cprops = cidx.size * proportions
        cprops = cprops.astype(int)
        cprops[-1] = cidx.size - cprops[:-1].sum()
        # print(cidx)
        cx = X.iloc[cidx,:]
        cy = y[cidx]
        for xpart, ypart, sz in zip(xparts, yparts, cprops):
            xpart.append(cx.iloc[:sz,:])
            ypart.append(cy[:sz])
            cx, cy = cx.iloc[sz:,:], cy[sz:]
    xparts = [pd.concat(xpart) for xpart in xparts]
    yparts = [np.concatenate(ypart) for ypart in yparts]
    parts = xparts + yparts
    return parts


xtr = X_train # ref
xte = X_test # ref
xtr, xcv, xva, ytr, ycv, yva = (
    split_train_cv_test(xtr, ytr, proportions=(.7, .15, .15))
)
                      

In [33]:
xtr.columns.tolist()

['amount_tsh',
 'gps_height',
 'wpt_name',
 'population',
 'public_meeting',
 'permit',
 'dry_season',
 'rainy_season',
 'age',
 'amount_per_man',
 0,
 1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 35,
 36,
 37,
 38,
 39,
 40,
 41,
 42,
 43,
 44,
 45,
 46,
 47,
 48,
 49,
 50,
 51,
 52,
 53,
 54,
 55,
 56,
 57,
 58,
 59,
 60,
 61,
 62,
 63,
 64,
 65,
 66,
 67,
 68,
 69,
 70,
 71,
 72]

In [34]:
display(xtr.shape, xte.shape)

(41578, 83)

(14850, 83)

In [35]:
# cat_features = np.array([xtr.columns.get_loc(c) for c in text_features if c in xtr.columns])
cat_features = []
sel = Pipeline([
    ('varthres', VarianceThreshold(1e-5)),
    ('main', SelectFromModel(RandomForestClassifier(n_estimators=300), 1e-3))
])
xtr = sel.fit_transform(xtr, ytr)
xcv = sel.transform(xcv)
xva = sel.transform(xva)
xte = sel.transform(xte)

display(xtr.shape,
        sel.steps[-1][-1].estimator_.feature_importances_)

(41578, 82)

array([ 0.0283357 ,  0.15690447,  0.08752522,  0.01992851,  0.02061537,
        0.01213976,  0.00701197,  0.09578512,  0.0409219 ,  0.0018183 ,
        0.00366012,  0.00259925,  0.00346457,  0.00396045,  0.00336118,
        0.00487899,  0.00205722,  0.00568783,  0.0035555 ,  0.00286629,
        0.00369217,  0.00450737,  0.00365075,  0.00150408,  0.00119483,
        0.0015581 ,  0.00276064,  0.00205894,  0.01885009,  0.00496537,
        0.00357313,  0.00606428,  0.00364655,  0.00342357,  0.00731001,
        0.00347477,  0.00413585,  0.0019757 ,  0.00207987,  0.00564675,
        0.00223268,  0.00481584,  0.00115047,  0.00190076,  0.00286424,
        0.00263339,  0.00255159,  0.00154814,  0.00137   ,  0.00645658,
        0.00555077,  0.00223158,  0.00231322,  0.00347965,  0.00790328,
        0.00204815,  0.00185058,  0.00199424,  0.01492843,  0.00194679,
        0.00873073,  0.00814269,  0.01509793,  0.02906389,  0.00174816,
        0.00919277,  0.0090173 ,  0.00168471,  0.00738867,  0.00

In [36]:
#X_train.to_csv('X_train_prepr2.csv')
#X_test.to_csv('X_test_prepr2.csv')
# pd.DataFrame(ytr, columns=y_enc1.classes_).to_csv('y_train_prepr2.csv')
#pd.Series(ytr).to_csv('y_train_prepr2.csv')
#pd.Series(y_enc1.classes_).to_csv('y_classes2.csv')

In [37]:
y_bin = sklearn.preprocessing.LabelBinarizer().fit(ytr)

ytr = y_bin.transform(ytr)
ycv = y_bin.transform(ycv)
yva = y_bin.transform(yva)

xscaler = StandardScaler().fit(xte)
xtr = xscaler.transform(xtr)
xte = xscaler.transform(xte)
xcv = xscaler.transform(xcv)
xva = xscaler.transform(xva)

def y_to_df(y, ids):
    y = y_bin.inverse_transform(y)
    y = y_enc1.inverse_transform(y)
    df = pd.DataFrame({'status_group': y}, index=ids)
    df.index.name = 'id'
    return df

In [38]:
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
import keras.utils

Using TensorFlow backend.


In [39]:

clf = Sequential([
    Dense(2*xtr.shape[1], input_shape=(xtr.shape[1],), activation='softmax'),
    # Dropout(.05),
    Dense(2*xtr.shape[1], activation='softmax'),
    Dense(2*xtr.shape[1], activation='softmax'),
    Dense(3, activation='softmax'),
])

In [40]:
clf.compile(optimizer='rmsprop',
          loss='categorical_crossentropy',
          metrics=['accuracy'])

In [41]:
clf.fit(xtr, ytr,
      batch_size=256,
      epochs=2500)

Epoch 1/2500
41578/41578 [==============================] - 1s - loss: 1.0184 - acc: 0.5431     
Epoch 2/2500
41578/41578 [==============================] - 1s - loss: 0.9312 - acc: 0.5431     
Epoch 3/2500
41578/41578 [==============================] - 1s - loss: 0.8973 - acc: 0.5431     
Epoch 4/2500
41578/41578 [==============================] - 1s - loss: 0.8901 - acc: 0.5431     
Epoch 5/2500
41578/41578 [==============================] - 1s - loss: 0.8896 - acc: 0.5431     
Epoch 6/2500
41578/41578 [==============================] - 1s - loss: 0.8896 - acc: 0.5431     
Epoch 7/2500
41578/41578 [==============================] - 1s - loss: 0.8897 - acc: 0.5431     
Epoch 8/2500
41578/41578 [==============================] - 1s - loss: 0.8896 - acc: 0.5431     
Epoch 9/2500
41578/41578 [==============================] - 1s - loss: 0.8896 - acc: 0.5431     
Epoch 10/2500
41578/41578 [==============================] - 1s - loss: 0.8896 - acc: 0.5431     
Epoch 11/2500
41578/41578 [==

41578/41578 [==============================] - 1s - loss: 0.5952 - acc: 0.7771     
Epoch 85/2500
41578/41578 [==============================] - 1s - loss: 0.5940 - acc: 0.7774     
Epoch 86/2500
41578/41578 [==============================] - 1s - loss: 0.5935 - acc: 0.7782     
Epoch 87/2500
41578/41578 [==============================] - 1s - loss: 0.5925 - acc: 0.7788     
Epoch 88/2500
41578/41578 [==============================] - 1s - loss: 0.5921 - acc: 0.7787     
Epoch 89/2500
41578/41578 [==============================] - 1s - loss: 0.5911 - acc: 0.7791     
Epoch 90/2500
41578/41578 [==============================] - 1s - loss: 0.5904 - acc: 0.7785     
Epoch 91/2500
41578/41578 [==============================] - 1s - loss: 0.5900 - acc: 0.7790     
Epoch 92/2500
41578/41578 [==============================] - 1s - loss: 0.5892 - acc: 0.7793     
Epoch 93/2500
41578/41578 [==============================] - 1s - loss: 0.5884 - acc: 0.7796     
Epoch 94/2500
41578/41578 [=======

41578/41578 [==============================] - 1s - loss: 0.5550 - acc: 0.7912     
Epoch 168/2500
41578/41578 [==============================] - 1s - loss: 0.5547 - acc: 0.7911     
Epoch 169/2500
41578/41578 [==============================] - 2s - loss: 0.5542 - acc: 0.7915     
Epoch 170/2500
41578/41578 [==============================] - 2s - loss: 0.5542 - acc: 0.7910     
Epoch 171/2500
41578/41578 [==============================] - 1s - loss: 0.5535 - acc: 0.7915     
Epoch 172/2500
41578/41578 [==============================] - 1s - loss: 0.5531 - acc: 0.7910     
Epoch 173/2500
41578/41578 [==============================] - 1s - loss: 0.5528 - acc: 0.7930     
Epoch 174/2500
41578/41578 [==============================] - 1s - loss: 0.5527 - acc: 0.7916     
Epoch 175/2500
41578/41578 [==============================] - 2s - loss: 0.5522 - acc: 0.7909     
Epoch 176/2500
41578/41578 [==============================] - 1s - loss: 0.5518 - acc: 0.7921     
Epoch 177/2500
41578/4157

41578/41578 [==============================] - 1s - loss: 0.5312 - acc: 0.7976     
Epoch 250/2500
41578/41578 [==============================] - 2s - loss: 0.5306 - acc: 0.7969     
Epoch 251/2500
41578/41578 [==============================] - 1s - loss: 0.5300 - acc: 0.7966     
Epoch 252/2500
41578/41578 [==============================] - 1s - loss: 0.5290 - acc: 0.7979     
Epoch 253/2500
41578/41578 [==============================] - 2s - loss: 0.5286 - acc: 0.7963     
Epoch 254/2500
41578/41578 [==============================] - 2s - loss: 0.5274 - acc: 0.7990     
Epoch 255/2500
41578/41578 [==============================] - 1s - loss: 0.5265 - acc: 0.8001     
Epoch 256/2500
41578/41578 [==============================] - 1s - loss: 0.5258 - acc: 0.7999     
Epoch 257/2500
41578/41578 [==============================] - 1s - loss: 0.5246 - acc: 0.8009     
Epoch 258/2500
41578/41578 [==============================] - 1s - loss: 0.5240 - acc: 0.8018     
Epoch 259/2500
41578/4157

41578/41578 [==============================] - 1s - loss: 0.4956 - acc: 0.8086     
Epoch 332/2500
41578/41578 [==============================] - 1s - loss: 0.4960 - acc: 0.8091     
Epoch 333/2500
41578/41578 [==============================] - 2s - loss: 0.4956 - acc: 0.8088     
Epoch 334/2500
41578/41578 [==============================] - 1s - loss: 0.4953 - acc: 0.8092     
Epoch 335/2500
41578/41578 [==============================] - 1s - loss: 0.4948 - acc: 0.8089     
Epoch 336/2500
41578/41578 [==============================] - 1s - loss: 0.4948 - acc: 0.8103     
Epoch 337/2500
41578/41578 [==============================] - 1s - loss: 0.4943 - acc: 0.8101     
Epoch 338/2500
41578/41578 [==============================] - 1s - loss: 0.4945 - acc: 0.8091     
Epoch 339/2500
41578/41578 [==============================] - 1s - loss: 0.4939 - acc: 0.8100     
Epoch 340/2500
41578/41578 [==============================] - 1s - loss: 0.4939 - acc: 0.8097     
Epoch 341/2500
41578/4157

...

41578/41578 [==============================] - 1s - loss: 0.4115 - acc: 0.8356     
Epoch 2135/2500
41578/41578 [==============================] - 1s - loss: 0.4118 - acc: 0.8363     
Epoch 2136/2500
41578/41578 [==============================] - 1s - loss: 0.4114 - acc: 0.8363     
Epoch 2137/2500
41578/41578 [==============================] - 1s - loss: 0.4115 - acc: 0.8369     
Epoch 2138/2500
41578/41578 [==============================] - 1s - loss: 0.4112 - acc: 0.8365     
Epoch 2139/2500
41578/41578 [==============================] - 1s - loss: 0.4120 - acc: 0.8361     
Epoch 2140/2500
41578/41578 [==============================] - 1s - loss: 0.4115 - acc: 0.8365     
Epoch 2141/2500
41578/41578 [==============================] - 1s - loss: 0.4117 - acc: 0.8369     
Epoch 2142/2500
41578/41578 [==============================] - 1s - loss: 0.4114 - acc: 0.8366     
Epoch 2143/2500
41578/41578 [==============================] - 1s - loss: 0.4114 - acc: 0.8368     
Epoch 2144/2500


41578/41578 [==============================] - 1s - loss: 0.4112 - acc: 0.8371     
Epoch 2217/2500
41578/41578 [==============================] - 1s - loss: 0.4114 - acc: 0.8376     
Epoch 2218/2500
41578/41578 [==============================] - 1s - loss: 0.4109 - acc: 0.8371     
Epoch 2219/2500
41578/41578 [==============================] - 1s - loss: 0.4113 - acc: 0.8369     
Epoch 2220/2500
41578/41578 [==============================] - 1s - loss: 0.4108 - acc: 0.8379     
Epoch 2221/2500
41578/41578 [==============================] - 1s - loss: 0.4108 - acc: 0.8366     
Epoch 2222/2500
41578/41578 [==============================] - 1s - loss: 0.4112 - acc: 0.8371     
Epoch 2223/2500
41578/41578 [==============================] - 1s - loss: 0.4108 - acc: 0.8368     
Epoch 2224/2500
41578/41578 [==============================] - 1s - loss: 0.4108 - acc: 0.8365     
Epoch 2225/2500
41578/41578 [==============================] - 1s - loss: 0.4104 - acc: 0.8368     
Epoch 2226/2500


41578/41578 [==============================] - 1s - loss: 0.4098 - acc: 0.8378     
Epoch 2299/2500
41578/41578 [==============================] - 1s - loss: 0.4100 - acc: 0.8373     
Epoch 2300/2500
41578/41578 [==============================] - 1s - loss: 0.4102 - acc: 0.8369     
Epoch 2301/2500
41578/41578 [==============================] - 1s - loss: 0.4100 - acc: 0.8371     
Epoch 2302/2500
41578/41578 [==============================] - 1s - loss: 0.4099 - acc: 0.8372     
Epoch 2303/2500
41578/41578 [==============================] - 1s - loss: 0.4102 - acc: 0.8374     
Epoch 2304/2500
41578/41578 [==============================] - 1s - loss: 0.4099 - acc: 0.8373     
Epoch 2305/2500
41578/41578 [==============================] - 1s - loss: 0.4103 - acc: 0.8368     
Epoch 2306/2500
41578/41578 [==============================] - 1s - loss: 0.4103 - acc: 0.8370     
Epoch 2307/2500
41578/41578 [==============================] - 1s - loss: 0.4105 - acc: 0.8372     
Epoch 2308/2500


41578/41578 [==============================] - 1s - loss: 0.4094 - acc: 0.8378     
Epoch 2381/2500
41578/41578 [==============================] - 1s - loss: 0.4094 - acc: 0.8379     
Epoch 2382/2500
41578/41578 [==============================] - 1s - loss: 0.4088 - acc: 0.8376     
Epoch 2383/2500
41578/41578 [==============================] - 1s - loss: 0.4091 - acc: 0.8377     
Epoch 2384/2500
41578/41578 [==============================] - 1s - loss: 0.4085 - acc: 0.8376     
Epoch 2385/2500
41578/41578 [==============================] - 1s - loss: 0.4086 - acc: 0.8380     
Epoch 2386/2500
41578/41578 [==============================] - 1s - loss: 0.4095 - acc: 0.8374     
Epoch 2387/2500
41578/41578 [==============================] - 1s - loss: 0.4087 - acc: 0.8379     
Epoch 2388/2500
41578/41578 [==============================] - 1s - loss: 0.4090 - acc: 0.8380     
Epoch 2389/2500
41578/41578 [==============================] - 1s - loss: 0.4088 - acc: 0.8385     
Epoch 2390/2500


41578/41578 [==============================] - 1s - loss: 0.4079 - acc: 0.8386     
Epoch 2463/2500
41578/41578 [==============================] - 1s - loss: 0.4078 - acc: 0.8391     
Epoch 2464/2500
41578/41578 [==============================] - 1s - loss: 0.4079 - acc: 0.8380     
Epoch 2465/2500
41578/41578 [==============================] - 1s - loss: 0.4083 - acc: 0.8380     
Epoch 2466/2500
41578/41578 [==============================] - 1s - loss: 0.4084 - acc: 0.8392     
Epoch 2467/2500
41578/41578 [==============================] - 1s - loss: 0.4085 - acc: 0.8382     
Epoch 2468/2500
41578/41578 [==============================] - 1s - loss: 0.4078 - acc: 0.8383     
Epoch 2469/2500
41578/41578 [==============================] - 1s - loss: 0.4081 - acc: 0.8374     
Epoch 2470/2500
41578/41578 [==============================] - 1s - loss: 0.4080 - acc: 0.8383     
Epoch 2471/2500
41578/41578 [==============================] - 1s - loss: 0.4080 - acc: 0.8379     
Epoch 2472/2500


In [47]:
yva_pred = clf.predict_classes(xva)

7712/8914 [========================>.....] - ETA: 0s

In [48]:
yva_pred

array([0, 2, 0, ..., 2, 2, 0])

In [50]:
sklearn.metrics.accuracy_score(y_bin.inverse_transform(yva), yva_pred)

0.75398249943908457

In [43]:
yte = clf.predict(xte)

In [ ]:
y_enc1 y_bin.inverse_transform(yte)